In [1]:
import numpy as np
import requests
import pandas as pd
from datetime import datetime
import mysql.connector
#import schedule
import time

## Connecting to MariaDB

In [ ]:
# Vivek's configuration
config = {
  'user': 'newuser',
  'password': 'newpassword',
  'host': '0.0.0.0',
  'database': 'test',
  'raise_on_warnings': True
}

cnx = mysql.connector.connect(**config)

cnx.close()

In [2]:
#Kristin's/Yabra's configuration
cnx = mysql.connector.connect(user='warehousing_team', password='nandan123', database='test', host = '172.17.0.3')
cursor = cnx.cursor()

## 311 Calls

In [3]:
#get data through API
with open("app_token.txt") as file: app_token = file.readline().rstrip()

# HTTPS address to access the 311 data
url = "https://data.cityofnewyork.us/resource/erm2-nwe9.json?"
querry = "$where=created_date between '2019-11-02T00:00:00' and '2019-11-09T00:00:00'&$limit=50000"
querry_url = url + "$$app_token=" + app_token + "&" + querry
response = requests.get(querry_url)
calls_311 = response.json()

In [4]:
calls_311[-1]

{'unique_key': '44891423',
 'created_date': '2019-11-08T23:58:55.000',
 'closed_date': '2019-11-09T09:09:36.000',
 'agency': 'NYPD',
 'agency_name': 'New York City Police Department',
 'complaint_type': 'Abandoned Vehicle',
 'descriptor': 'With License Plate',
 'location_type': 'Street/Sidewalk',
 'incident_zip': '11379',
 'incident_address': '77-23 PENELOPE AVENUE',
 'street_name': 'PENELOPE AVENUE',
 'cross_street_1': '77 STREET',
 'cross_street_2': '77 PLACE',
 'intersection_street_1': '77 STREET',
 'intersection_street_2': '77 PLACE',
 'city': 'MIDDLE VILLAGE',
 'landmark': 'PENELOPE AVENUE',
 'status': 'Closed',
 'resolution_description': 'The Police Department responded to the complaint and with the information available observed no evid',
 'resolution_action_updated_date': '2019-11-09T14:09:38.000',
 'community_board': '05 QUEENS',
 'borough': 'QUEENS',
 'x_coordinate_state_plane': '1018291',
 'y_coordinate_state_plane': '200979',
 'open_data_channel_type': 'MOBILE',
 'park_faci

In [5]:
#Select keys and extract appropriate data
keys_311 = ['unique_key', 
            'created_date',
            'closed_date',
            'agency',
            'agency_name',
            'complaint_type',
            'descriptor',
            'incident_zip',
            'borough',
           'facility_type',
           'location_type',
            'status',
           'due_date',
            'resolution_action_updated_date',
            'x_coordinate_state_plane',
            'y_coordinate_state_plane',
            'open_data_channel_type',
            'latitude',
            'longitude']

data_311 = []
for call in calls_311:
    dict_calls = {'unique_key': call['unique_key']}
    for key in keys_311:
       if np.sum([i==key for i in call.keys()])==1: 
           dict_calls[key] = call[key]
       else:
            dict_calls[key] = None
    data_311.append(dict_calls)

In [6]:
#cnx = mysql.connector.connect(**config)
#cursor = cnx.cursor()

calls_sql = (
   "insert into 311_calls"
   "(unique_key, created_date, closed_date, agency, agency_name, complaint_type,"
            "descriptor, location_type, incident_zip, borough, facility_type,"
            "status, due_date, resolution_action_updated_date, x_coordinate_state_plane,"
            "y_coordinate_state_plane, open_data_channel_type, latitude, longitude)"
   "values (%(unique_key)s, %(created_date)s, %(closed_date)s, %(agency)s, %(agency_name)s, %(complaint_type)s,"
            "%(descriptor)s, %(location_type)s, %(incident_zip)s, %(borough)s, %(facility_type)s,"
            "%(status)s, %(due_date)s, %(resolution_action_updated_date)s, %(x_coordinate_state_plane)s,"
            "%(y_coordinate_state_plane)s, %(open_data_channel_type)s, %(latitude)s, %(longitude)s)")

for call in data_311:
   cursor.execute(calls_sql, call)

cnx.commit()
#cursor.close()
#cnx.close()

## Accidents

In [14]:
# 1. Get the data through API
with open("app_token.txt") as file: app_token = file.readline().rstrip()

# HTTPS address to access the 311 data
url_acc = "https://data.cityofnewyork.us/resource/h9gi-nx95.json?"
querry_acc = "$where=accident_date > '2019-01-01T00:00:00'&$limit=50000"
querry_url_acc = url_acc + "$$app_token=" + app_token + "&" + querry_acc
response = requests.get(querry_url_acc)
acc = response.json()

In [17]:
acc[20]

{'accident_date': '2019-01-02T00:00:00.000',
 'accident_time': '2:30',
 'latitude': '40.8957250',
 'longitude': '-73.8748200',
 'location': {'type': 'Point', 'coordinates': [-73.87482, 40.895725]},
 'on_street_name': 'EAST 233 STREET                 ',
 'number_of_persons_injured': '1',
 'number_of_persons_killed': '0',
 'number_of_pedestrians_injured': '0',
 'number_of_pedestrians_killed': '0',
 'number_of_cyclist_injured': '0',
 'number_of_cyclist_killed': '0',
 'number_of_motorist_injured': '1',
 'number_of_motorist_killed': '0',
 'contributing_factor_vehicle_1': 'Driver Inattention/Distraction',
 'contributing_factor_vehicle_2': 'Unspecified',
 'collision_id': '4060646',
 'vehicle_type_code1': 'Station Wagon/Sport Utility Vehicle',
 'vehicle_type_code2': 'Sedan'}

In [18]:
# 2.Select keys and extract appropriate data
keys_acc =  ["collision_id", "accident_date", "accident_time", "borough", "zip_code", "latitude", "longitude", 
             "number_of_persons_injured", "number_of_persons_killed", 
             "number_pedestrians_injured", "number_of_pedestrians_killed", 
             "number_of_cyclists_injured", "number_of_cyclists_killed", 
             "number_of_motorists_injured", "number_of_motorists_killed", 
             "contributing_factor_vehicle_1", "contributing_factor_vehicle_2"]
data_acc = []
for a in acc:
    #start by adding the collission_id to the dictionary
    dict_acc = {'collision_id': a['collision_id']}
    for key in keys_acc:
       if np.sum([k == key for k in a.keys()])== 1: 
           dict_acc[key] = a[key]
       else:
            dict_acc[key] = None
    data_acc.append(dict_acc)

In [19]:
len(data_acc[0])

17

In [20]:
data_acc[-1]

{'collision_id': '4108751',
 'accident_date': '2019-03-31T00:00:00.000',
 'accident_time': '4:00',
 'borough': None,
 'zip_code': None,
 'latitude': '40.8793200',
 'longitude': '-73.8698040',
 'number_of_persons_injured': '0',
 'number_of_persons_killed': '0',
 'number_pedestrians_injured': None,
 'number_of_pedestrians_killed': '0',
 'number_of_cyclists_injured': None,
 'number_of_cyclists_killed': None,
 'number_of_motorists_injured': None,
 'number_of_motorists_killed': None,
 'contributing_factor_vehicle_1': 'Fell Asleep',
 'contributing_factor_vehicle_2': None}

In [21]:
# 3. Get data into MariaDB table

#Define the insert querry
acc_sql = (
   "insert into accidents"
   "(collision_id, accident_date, accident_time, borough, zip_code, latitude, longitude,"
    "number_of_persons_injured, number_of_persons_killed, " 
    "number_pedestrians_injured, number_of_pedestrians_killed, " 
    "number_of_cyclists_injured, number_of_cyclists_killed, " 
    "number_of_motorists_injured, number_of_motorists_killed, " 
    "contributing_factor_vehicle_1, contributing_factor_vehicle_2)"
   
    "values (%(collision_id)s, %(accident_date)s, %(accident_time)s, %(borough)s, %(zip_code)s, %(latitude)s,"
    "%(longitude)s, %(number_of_persons_injured)s, %(number_of_persons_killed)s," 
    "%(number_pedestrians_injured)s, %(number_of_pedestrians_killed)s, %(number_of_cyclists_injured)s," 
    "%(number_of_cyclists_killed)s, %(number_of_motorists_injured)s, %(number_of_motorists_killed)s,"
    "%(contributing_factor_vehicle_1)s, %(contributing_factor_vehicle_2)s)")

#Execute the querry
for a in data_acc:
   cursor.execute(acc_sql, a)

In [22]:
# 4. Commit changes and close connection with the database
cnx.commit()
cursor.close()
cnx.close()

## Schedualing

In [ ]:
def job():
    print("I'm working...")
    
schedule.every(2).seconds.do(job)

# schedule.every(10).minutes.do(job)
# schedule.every().hour.do(job)
# schedule.every().day.at("10:30").do(job)
# schedule.every(5).to(10).minutes.do(job)
# schedule.every().monday.do(job)
# schedule.every().wednesday.at("13:15").do(job)
# schedule.every().minute.at(":17").do(job)

while True:
    schedule.run_pending()
    time.sleep(1)
#sometimes the job runs more than once (???)